In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import StratifiedKFold


In [2]:
def collect_and_label(dataset): 
    

    df = pd.read_csv(dataset)
    alpha = df['scp_codes'].str.split("'").str[1].str[-2:]=='MI'  
    beta = df['scp_codes'].str.split("'").str[1]=='NORM'       
    df = df[alpha|beta]
    df['label'] = df['scp_codes'].str.split("'").str[1]      
      
    inst_c1 = df[df['label'] == 'NORM']
    inst_c1 = inst_c1.sample(n = 100, random_state = 1)
    
    inst_c2 = df[df['label'] == 'ILMI']
    inst_c2 = inst_c2.sample(n = 100, random_state = 1)
    

    df_new = pd.concat([inst_c1, inst_c2], ignore_index = True)
    df_new = df_new.sample(frac = 1, random_state=42)
    
    return  df_new

In [3]:
def div_and_label(dataframe):  
    
    X = dataframe[['ecg_id', 'filename_hr']]   
    X = X.to_numpy()
    y = dataframe['label']
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_y = encoder.transform(y)
    
    return (X, encoded_y)

In [4]:
alpha = collect_and_label("C:/Users/Pushpam/Downloads/ptbxl_database.csv")  
gamma0, gamma1 = div_and_label(alpha)
gamma0 = gamma0[:,1]

print(gamma0.shape)
print(gamma1.shape)

(200,)
(200,)


In [5]:
print(gamma0)

['records500/08000/08190_hr' 'records500/12000/12365_hr'
 'records500/03000/03403_hr' 'records500/18000/18438_hr'
 'records500/15000/15664_hr' 'records500/00000/00528_hr'
 'records500/00000/00548_hr' 'records500/16000/16405_hr'
 'records500/01000/01950_hr' 'records500/18000/18043_hr'
 'records500/02000/02897_hr' 'records500/19000/19404_hr'
 'records500/17000/17165_hr' 'records500/18000/18860_hr'
 'records500/19000/19842_hr' 'records500/07000/07481_hr'
 'records500/10000/10582_hr' 'records500/14000/14777_hr'
 'records500/14000/14694_hr' 'records500/20000/20903_hr'
 'records500/00000/00742_hr' 'records500/11000/11204_hr'
 'records500/13000/13228_hr' 'records500/04000/04262_hr'
 'records500/09000/09469_hr' 'records500/00000/00543_hr'
 'records500/14000/14177_hr' 'records500/06000/06152_hr'
 'records500/13000/13157_hr' 'records500/06000/06025_hr'
 'records500/12000/12061_hr' 'records500/07000/07628_hr'
 'records500/01000/01321_hr' 'records500/10000/10221_hr'
 'records500/05000/05889_hr' 'r

In [6]:
gamma1

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0])

In [7]:
from scipy import signal
import matplotlib.pyplot as plt

In [8]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
from scipy.signal import butter, filtfilt, detrend

directory = 'D:/Internship/MIDataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1'

X = []
for itr in range(gamma0.shape[0]):
    record_name = str(gamma0[itr])
    
    signal, meta_val = wfdb.rdsamp(directory + '/' + record_name)
    value = signal.T
    ecg_signals = value

    X.append(detrend(ecg_signals))
    

In [9]:
X = np.array(X)
print(X.shape)

(200, 12, 5000)


In [10]:
y_data = gamma1;
print(y_data.shape)

(200,)


In [12]:
# windowing X in xnew
xnew = []
ynew = []
patient_ids = []
for i in range(X.shape[0]):
    xnew.append(X[i,:,0:1000])
    xnew.append(X[i,:,1000:2000])
    xnew.append(X[i,:,2000:3000])
    xnew.append(X[i,:,3000:4000])
    xnew.append(X[i,:,4000:5000])
    for j in range(5):
        ynew.append(y_data[i])
        patient_ids.append(i)
X = np.array(xnew)
y_data = np.array(ynew)
patient_ids = np.array(patient_ids)

In [11]:
from dtaidistance import dtw
d = dtw.distance_matrix_fast(X[0])

In [12]:
print(d.shape)

(12, 12)


In [ ]:
# distance = np.load('distance_ILMI_1000+393_not_random_dtw_matrix.npy')
# y_data = np.load('y_data_ILMI_1000+393_not_random_dtw_matrix.npy')
# patient_ids = np.load('patient_ids_ILMI_1000+393_not_random_dtw_matrix.npy')

In [13]:
from dtaidistance import dtw
distance = np.zeros((X.shape[0],12,12))
for i in range(X.shape[0]):
        distance[i] = dtw.distance_matrix_fast(X[i])
        print(distance[i])
        print(i)

[[ 0.          3.34842819  5.27840328  7.47589329  4.18838655  3.12302622
   7.00162723  8.66918685  2.98648604  2.89516904  3.47461609  2.66439589]
 [ 3.34842819  0.          6.39949676  9.75678211  7.02714     2.81209182
   9.6098373  11.12774959  5.87353633  2.86653116  2.95991843  3.2295338 ]
 [ 5.27840328  6.39949676  0.          8.05914421  2.73808776  2.77663694
   6.91701232 10.53302471  4.41278547  7.31002628  7.82256022  6.05931275]
 [ 7.47589329  9.75678211  8.05914421  0.          5.95542466  8.35823285
   2.95201217  3.50985963  4.96632005  9.46972746 10.52251601  9.40136978]
 [ 4.18838655  7.02714     2.73808776  5.95542466  0.          4.01764622
   4.91652039  8.29048878  1.96949082  7.03252439  7.65790842  6.01987354]
 [ 3.12302622  2.81209182  2.77663694  8.35823285  4.01764622  0.
   7.6425078  10.54484472  4.0653292   4.07951787  4.55615221  3.48188398]
 [ 7.00162723  9.6098373   6.91701232  2.95201217  4.91652039  7.6425078
   0.          3.1198947   3.94041887  9.

[[ 0.          3.38062795  9.26669001  8.50231862  2.35880708  5.62676624
  18.97388359 23.40651331 16.20618704  6.27463712  2.24124148  2.43271202]
 [ 3.38062795  0.          8.94126505  7.05338401  4.44752741  4.33997174
  18.88672241 24.23176225 16.88558726  9.07502746  4.2105422   1.72686797]
 [ 9.26669001  8.94126505  0.          8.46655729  8.39380138  3.99020085
   5.96040953  9.95527193  6.98018073  9.31420721  8.77538757  9.70759702]
 [ 8.50231862  7.05338401  8.46655729  0.          8.77842263  6.29866969
  14.67778704 19.54417698 12.96498075 11.81533063 11.48604805  8.49835811]
 [ 2.35880708  4.44752741  8.39380138  8.77842263  0.          5.54027577
  16.66735318 20.71874599 13.70411172  4.40409745  2.45148662  3.55144816]
 [ 5.62676624  4.33997174  3.99020085  6.29866969  5.54027577  0.
  11.66365097 16.38956312 10.58813148  7.21074796  5.54209931  5.43633686]
 [18.97388359 18.88672241  5.96040953 14.67778704 16.66735318 11.66365097
   0.          4.90289978  5.44201552 16

[[ 0.          3.87599293  4.2676149   6.41518993  2.04962909  5.58690782
   7.03225943  4.20428852  2.89036838  3.06147926  3.16573702 15.25792775]
 [ 3.87599293  0.          2.84602981  5.69562203  3.81653165  1.93460852
   9.00017763  3.42663606  3.02649731  3.64162873  2.19400114 14.93040806]
 [ 4.2676149   2.84602981  0.          4.26084098  4.43285874  3.17180242
   7.64036973  3.85929894  3.25399624  4.45965454  3.09060873 14.13166338]
 [ 6.41518993  5.69562203  4.26084098  0.          4.44095339  6.03442988
   5.12115309  4.09602345  5.18080654  6.86359241  5.97425404 16.59580297]
 [ 2.04962909  3.81653165  4.43285874  4.44095339  0.          5.02613623
   5.69492185  3.18893816  2.95817958  3.80212238  3.44729628 15.67602039]
 [ 5.58690782  1.93460852  3.17180242  6.03442988  5.02613623  0.
   9.7088825   3.42228568  3.29995149  4.86190781  2.91960961 15.46795395]
 [ 7.03225943  9.00017763  7.64036973  5.12115309  5.69492185  9.7088825
   0.          7.58822291  6.71160813  7.

[[0.         5.51643541 7.36438509 5.72287317 1.95370101 6.905952
  4.35634269 5.36728323 4.02681077 5.40406224 5.83086148 6.5515549 ]
 [5.51643541 0.         3.3907704  3.93378255 5.27694997 2.64955246
  7.41028067 3.67527053 3.82752002 2.05003941 1.93678446 2.18662542]
 [7.36438509 3.3907704  0.         4.96659619 7.60236332 2.28185762
  9.46745597 7.3431386  6.77306046 3.23906001 3.03109408 3.67842986]
 [5.72287317 3.93378255 4.96659619 0.         5.13577882 4.18210374
  7.06197061 4.41495812 4.51409248 4.03484239 4.08765941 4.07962591]
 [1.95370101 5.27694997 7.60236332 5.13577882 0.         6.88528821
  2.79312754 3.70183731 3.86658055 5.73268278 6.08953246 6.27212816]
 [6.905952   2.64955246 2.28185762 4.18210374 6.88528821 0.
  9.74524892 5.57858464 4.93319403 2.40078109 2.19491347 1.92063311]
 [4.35634269 7.41028067 9.46745597 7.06197061 2.79312754 9.74524892
  0.         5.81450278 5.2586924  8.45773    8.54091131 8.8086554 ]
 [5.36728323 3.67527053 7.3431386  4.41495812 3.701

[[ 0.          3.08580675 11.07445828 12.36922244  2.48367541  8.33077326
  16.01726436 14.2916852  11.24492934  7.36923828  6.75768238  1.69927309]
 [ 3.08580675  0.          7.31847848  8.65248443  3.12175375  4.0736992
  12.79799085 12.18610282  9.86560403  6.5757818   9.48034935  3.02401093]
 [11.07445828  7.31847848  0.          2.52465485  8.9230604   3.10145451
   5.39683568  5.52387696  7.10071426 12.4861721  17.84453277 11.14196044]
 [12.36922244  8.65248443  2.52465485  0.         10.49147314  4.89582414
   3.67693779  4.7635349   6.98920048 12.16654872 18.48032617 12.4488778 ]
 [ 2.48367541  3.12175375  8.9230604  10.49147314  0.          6.0132903
  14.81429427 13.43334476 10.4042961   9.24083088  9.7865024   3.0017149 ]
 [ 8.33077326  4.0736992   3.10145451  4.89582414  6.0132903   0.
   9.21921386  8.80637692  7.95353138 10.21058576 15.39271495  8.21746625]
 [16.01726436 12.79799085  5.39683568  3.67693779 14.81429427  9.21921386
   0.          4.15341281  6.53904495 12.6

...

[[ 0.         10.89983899 16.69750372 13.55601286  1.02956508 13.68526533
  20.36716283 27.24638122 19.8091734  12.16033314  4.64946674  5.7262591 ]
 [10.89983899  0.         11.1945261   5.35688541 10.59293662  4.6424049
  17.17883919 26.85878155 17.5882096  10.74526857  6.50725694  3.78033495]
 [16.69750372 11.1945261   0.          4.70961085 16.70147945  5.60310648
   4.33551279 13.60585502  6.75598185  5.07221497 10.56136081 12.47758158]
 [13.55601286  5.35688541  4.70961085  0.         13.38030529  1.99338702
   9.97826865 19.94222489 11.39750546  6.84512519  7.93182585  7.46663429]
 [ 1.02956508 10.59293662 16.70147945 13.38030529  0.         13.44123323
  20.52751788 27.6130391  19.98122167 12.36689173  4.68732464  5.34533111]
 [13.68526533  4.6424049   5.60310648  1.99338702 13.44123323  0.
  11.47685177 21.72734536 12.7997739   7.68717733  7.85541392  7.00577702]
 [20.36716283 17.17883919  4.33551279  9.97826865 20.52751788 11.47685177
   0.          8.00632371  3.76466287  5.

[[ 0.          3.71985351  5.24470241  4.90469097  2.50565548  5.21328826
   5.67296091  7.15045284 12.6110308   6.75478279  3.31326824  3.399411  ]
 [ 3.71985351  0.          3.37309966  4.62128954  3.98332804  2.72266188
   5.43956189  7.77941506 13.55540134  7.59203686  4.2947626   3.4530479 ]
 [ 5.24470241  3.37309966  0.          3.94012292  4.87790313  2.56755053
   4.29021781  7.21797822 13.34122619  8.06513637  5.11794251  4.38289186]
 [ 4.90469097  4.62128954  3.94012292  0.          4.14343318  4.66734354
   4.19853812  6.98136818 13.09575763  7.27001422  4.91609316  4.63992692]
 [ 2.50565548  3.98332804  4.87790313  4.14343318  0.          4.52196769
   5.38935774  7.15002479 12.75458467  6.44738621  2.97574942  2.53709118]
 [ 5.21328826  2.72266188  2.56755053  4.66734354  4.52196769  0.
   5.50369618  8.2960341  14.29573928  8.48813354  5.17202102  4.28267473]
 [ 5.67296091  5.43956189  4.29021781  4.19853812  5.38935774  5.50369618
   0.          5.68631847 11.54965128  6

[[ 0.          5.3514669   6.67035402  5.27310916  1.28763078  5.93526369
   2.96309336 12.84672562 15.61704799 16.70042371  4.8398479  19.08847887]
 [ 5.3514669   0.          4.21026997  3.03353659  5.52767344  1.92006502
   3.5003636  14.8935292  21.60551474 23.7398971  10.55449335 18.61277911]
 [ 6.67035402  4.21026997  0.          3.31159864  7.12661874  2.26786385
   6.24208988 13.96806756 21.63950437 24.09888045 11.23050385 19.12976344]
 [ 5.27310916  3.03353659  3.31159864  0.          4.77681675  3.03927199
   3.2492682  12.00217287 20.20096028 22.9817216   9.72803505 19.27586758]
 [ 1.28763078  5.52767344  7.12661874  4.77681675  0.          5.72876743
   2.60000504 13.99916524 16.7209355  17.08742487  5.17509882 19.28743161]
 [ 5.93526369  1.92006502  2.26786385  3.03927199  5.72876743  0.
   4.09469741 15.47581088 22.50514654 24.33886767 10.8649199  18.74311778]
 [ 2.96309336  3.5003636   6.24208988  3.2492682   2.60000504  4.09469741
   0.         14.22660085 19.22540292 20

[[ 0.          2.74304285  6.98762797  7.6530945   1.40051713  6.20376401
   7.58286261 13.03426407 14.27601059 13.61014025  7.23980788  1.29645594]
 [ 2.74304285  0.          3.81876813  5.08826036  1.99971379  2.80519124
   5.65198186 13.79660958 15.53998954 15.54622867  9.66473319  2.78941561]
 [ 6.98762797  3.81876813  0.          1.98996183  5.56920851  2.42652195
   4.55472945 14.93304293 17.05760859 18.16030873 13.63892865  7.19676886]
 [ 7.6530945   5.08826036  1.98996183  0.          6.32628961  4.34068676
   5.28323614 14.92781151 17.14218974 18.38947628 13.97007726  7.69793463]
 [ 1.40051713  1.99971379  5.56920851  6.32628961  0.          4.5389104
   7.22792754 14.0551065  15.50153201 15.14989996  8.93546963  1.95013189]
 [ 6.20376401  2.80519124  2.42652195  4.34068676  4.5389104   0.
   5.97852341 16.32777415 18.27094541 18.82889148 13.43563454  6.47945524]
 [ 7.58286261  5.65198186  4.55472945  5.28323614  7.22792754  5.97852341
   0.         10.61599089 12.95323316 14.

[[ 0.          2.95810011 11.07158364 12.78833342  2.36842797  7.81389082
  15.24069613  9.56493239  7.84457736  8.29061817  4.3639443   1.71250862]
 [ 2.95810011  0.          7.65813502 10.35818495  1.63122132  4.05967273
  14.0248556   8.28001762  8.86323786 12.11350523  8.17051056  3.90393483]
 [11.07158364  7.65813502  0.          3.544767    7.60457849  4.02835435
   8.38197251 10.96199403 16.35560814 21.21409136 17.20943023 12.71098769]
 [12.78833342 10.35818495  3.544767    0.         10.58895582  8.97006381
   4.82816876  9.66000421 15.15095535 21.1107318  17.94124775 14.0679851 ]
 [ 2.36842797  1.63122132  7.60457849 10.58895582  0.          4.38349647
  14.44932468  8.99538451  9.96557565 11.95839836  7.58919185  3.42422434]
 [ 7.81389082  4.05967273  4.02835435  8.97006381  4.38349647  0.
  14.10239376  9.58562575 13.36827519 18.09760931 13.98172993  9.32185208]
 [15.24069613 14.0248556   8.38197251  4.82816876 14.44932468 14.10239376
   0.          9.56667632 14.88175251 20

In [15]:
# np.save('distance_ILMI_100+100_random_dtw_matrix_no_windowing.npy',distance)
# np.save('y_data_ILMI_100+100_random_dtw_matrix_no_windowing.npy',y_data)

In [17]:
t = distance
print(t.shape)

(6965, 12, 12)


In [ ]:
# from scipy.signal import coherence
# from glob import glob
# import scipy.io as sio
# import scipy.signal as sig

# cross_corr_matrices_list = []
# coherence_matrices_list = []


# for patient_data in X:
#     cross_corr_matrix = np.zeros((12, 12))
#     coherence_matrix = np.zeros((12, 12))

    
#     for i in range(12):
#         for j in range(i, 12):
#             lead_i = patient_data[i]
#             lead_j = patient_data[j]
            
#             cross_corr = np.corrcoef(lead_i, lead_j)[0, 1]
            
#             f, coh = coherence(lead_i, lead_j)  
#             coherence_value = np.mean(coh)  # Storing the average coherence value
            
            
#             cross_corr_matrix[i, j] = cross_corr
#             cross_corr_matrix[j, i] = cross_corr
            
            
#             coherence_matrix[i, j] = coherence_value
#             coherence_matrix[j, i] = coherence_value      
    
    
#     cross_corr_matrices_list.append(cross_corr_matrix)  
#     coherence_matrices_list.append(coherence_matrix)
    




In [ ]:
# # def create_K(A_i, B_i, C_i):
# #     return np.concatenate((A_i, B_i, C_i), axis=1)
# def create_K(A_i, B_i, C_i, D_i):
#     top_row = np.concatenate((A_i, B_i), axis=1)
#     bottom_row = np.concatenate((C_i, D_i), axis=1)
#     return np.concatenate((top_row, bottom_row), axis=0)

# K_matrices=[]
# for i in range(X.shape[0]):
#     K_i = create_K(cross_corr_matrices_list[i],coherence_matrices_list[i],distance[i],distance[i])
#     K_matrices.append(K_i)

# t = np.array(K_matrices)
# print(t.shape)

In [ ]:
# t = np.array(t).reshape(distance.shape[0],24,24,1)
# print(t.shape)

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D

In [22]:
X_train, X_test, y_train, y_test = train_test_split(t, y_data,
                                                    test_size=0.25,
                                                    random_state=42)

K.clear_session()


model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())

# model.add(Dense(1024, activation='relu'))
# model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, batch_size=32,
          epochs=15, verbose=1, validation_split=0.25)
model.evaluate(X_test, y_test)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10, 10, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 5, 10)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 250)               0         
                                                                 
 dense (Dense)               (None, 256)               64256     
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                        

[0.22576433420181274, 0.9328358173370361]

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, y_data, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = y_data[train_idx], y_data[test_idx]

    tf.keras.backend.clear_session()

    model = Sequential()

    # Use 4 input channels
    model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=32,
                        epochs=15,
                        verbose=1,
                        validation_split=0.25)

    predictions = model.predict(X_test)
    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


Epoch 1/15
131/131 [==============================] - 2s 8ms/step - loss: 0.3840 - accuracy: 0.8544 - val_loss: 0.3062 - val_accuracy: 0.8880
Epoch 2/15
131/131 [==============================] - 1s 7ms/step - loss: 0.2570 - accuracy: 0.8980 - val_loss: 0.2845 - val_accuracy: 0.8959
Epoch 3/15
131/131 [==============================] - 1s 7ms/step - loss: 0.2162 - accuracy: 0.9155 - val_loss: 0.3103 - val_accuracy: 0.8650
Epoch 4/15
131/131 [==============================] - 1s 7ms/step - loss: 0.1891 - accuracy: 0.9291 - val_loss: 0.2854 - val_accuracy: 0.9031
Epoch 5/15
131/131 [==============================] - 1s 7ms/step - loss: 0.1707 - accuracy: 0.9397 - val_loss: 0.3120 - val_accuracy: 0.8995
Epoch 6/15
131/131 [==============================] - 1s 7ms/step - loss: 0.1435 - accuracy: 0.9490 - val_loss: 0.3633 - val_accuracy: 0.8823
Epoch 7/15
131/131 [==============================] - 1s 7ms/step - loss: 0.1119 - accuracy: 0.9569 - val_loss: 0.3561 - val_accuracy: 0.8981
Epoch 

Epoch 10/15
131/131 [==============================] - 1s 7ms/step - loss: 0.0799 - accuracy: 0.9684 - val_loss: 0.3855 - val_accuracy: 0.8917
Epoch 11/15
131/131 [==============================] - 1s 7ms/step - loss: 0.0636 - accuracy: 0.9763 - val_loss: 0.4638 - val_accuracy: 0.9010
Epoch 12/15
131/131 [==============================] - 1s 7ms/step - loss: 0.0680 - accuracy: 0.9761 - val_loss: 0.3376 - val_accuracy: 0.9096
Epoch 13/15
131/131 [==============================] - 1s 7ms/step - loss: 0.0400 - accuracy: 0.9842 - val_loss: 0.3468 - val_accuracy: 0.9082
Epoch 14/15
131/131 [==============================] - 1s 7ms/step - loss: 0.1020 - accuracy: 0.9658 - val_loss: 0.3619 - val_accuracy: 0.9053
Epoch 15/15
44/44 [==============================] - 0s 2ms/step
[[ 65   9]
 [ 15 189]]
Accuracy: 0.9136690647482014
Sensitivity: 0.9264705882352942
Specificity: 0.8783783783783784
Epoch 1/15
131/131 [==============================] - 2s 8ms/step - loss: 0.3590 - accuracy: 0.8529 - va

In [ ]:
# np.save('distance_ALMI_800+164_not_random_windowed_dtw_matrix.npy',np.array(distance))
# np.save('y_data_ALMI_800+164_not_random_wi`andowed_dtw_matrix.npy',np.array(y_data))

In [ ]:
# for i in range(12):
#         distance[:,i,i] = 0
# print(distance[0:1])

In [ ]:
# import numpy as np
# from scipy.signal import coherence

# cross_corr_matrices_list = []
# mae_matrices_list = []
# rmse_matrices_list = []
# coherence_matrices_list = []

# for patient_data in X:
#     cross_corr_matrix = np.zeros((12, 12))
#     mae_matrix = np.zeros((12, 12))
#     rmse_matrix = np.zeros((12, 12))
#     coherence_matrix = np.zeros((12, 12))

#     for i in range(12):
#         for j in range(i, 12):
#             lead_i = patient_data[i]
#             lead_j = patient_data[j]
            
#             cross_corr = np.corrcoef(lead_i, lead_j)[0, 1]
#             mae = np.mean(np.abs(lead_i - lead_j))
#             rmse = np.sqrt(np.mean((lead_i - lead_j) ** 2))
#             f, coh = coherence(lead_i, lead_j)  
#             coherence_value = np.mean(coh)  # Store the average coherence value
            
#             cross_corr_matrix[i, j] = cross_corr
#             cross_corr_matrix[j, i] = cross_corr
#             mae_matrix[i, j] = mae
#             mae_matrix[j, i] = mae
#             rmse_matrix[i, j] = rmse
#             rmse_matrix[j, i] = rmse
#             coherence_matrix[i, j] = coherence_value
#             coherence_matrix[j, i] = coherence_value
    
#     # Append the matrices to the corresponding lists
#     cross_corr_matrices_list.append(cross_corr_matrix)
#     mae_matrices_list.append(mae_matrix)
#     rmse_matrices_list.append(rmse_matrix)
#     coherence_matrices_list.append(coherence_matrix)

In [ ]:
# def create_K(A_i, B_i, C_i, D_i):
#     top_row = np.concatenate((A_i, B_i), axis=1)
#     bottom_row = np.concatenate((C_i, D_i), axis=1)
#     return np.concatenate((top_row, bottom_row), axis=0)

# K_matrices=[]
# for i in range(X.shape[0]):
#     K_i = create_K(cross_corr_matrices_list[i], mae_matrices_list[i], rmse_matrices_list[i], coherence_matrices_list[i])
#     K_matrices.append(K_i)

In [ ]:
# t = np.array(K_matrices).reshape(X.shape[0],24,24,1)
t = np.array(distance).reshape(X.shape[0],12,12,1)
print(t.shape)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(t, y_data,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D

In [ ]:
K.clear_session()


model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())

# model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='BinaryCrossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, batch_size=15,
          epochs=5, verbose=1, validation_split=0.25)
model.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
y_train[0:10]

In [ ]:
model.fit(X_train, y_train, batch_size=16,
          epochs=5, verbose=1, validation_split=0.25)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
np.count_nonzero(y_data[0:1000])

In [ ]:
# windowing X in xnew
# xnew = np.zeros((5*(X.shape[0]),12,1000))
xnew = []
ynew = []
patient_ids = []
for i in range(X.shape[0]):
    xnew.append(X[i,:,0:1000])
    xnew.append(X[i,:,1000:2000])
    xnew.append(X[i,:,2000:3000])
    xnew.append(X[i,:,3000:4000])
    xnew.append(X[i,:,4000:5000])
    for j in range(5):
        ynew.append(y_data[i])
        patient_ids.append(i)

In [ ]:
xnew = np.array(xnew)
ynew = np.array(ynew)
patient_ids = np.array(patient_ids)

In [ ]:
xnew.shape

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV

In [ ]:
rocket=Rocket()
# rocket=Rocket(num_kernels=4000)
rocket.fit(xnew)
xt=rocket.transform(xnew)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(xt, ynew,
#                                                     test_size=0.3,
#                                                     random_state=42)

In [ ]:
# classifier=SGDClassifier()
# classifier.fit(X_train,y_train)

In [ ]:
# Group-k-fold

from sklearn.model_selection import GroupKFold

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
for train_idx, test_idx in gkf.split(xt, ynew, groups=patient_ids):
    X_train, X_test = xt.iloc[train_idx], xt.iloc[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    classifier = SGDClassifier()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy
    print(confusion_matrix(y_test_majority, y_pred_majority))
    print(accuracy)

average_accuracy = sum_accuracy / 5
print("Average Accuracy:", average_accuracy)

In [ ]:
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import KFold
# kf=KFold(n_splits=5,shuffle=True)

In [ ]:
# sum=0
# for tr_idx, test_idx in kf.split(xt):
#     X_train, X_test=xt.iloc[tr_idx,:],xt.iloc[test_idx,:]
#     y_train, y_test=ynew[tr_idx],ynew[test_idx]

#     classifier=SGDClassifier()
#     classifier.fit(X_train,y_train)

#     y_pred=classifier.predict(X_test)
#     y_pred_majority = []
#     for i in range(0, y_data.shape[0], 5):
#         segment_predictions = y_pred[i:i+5]
#         majority_vote = np.bincount(segment_predictions).argmax()
#         y_pred_majority.append(majority_vote)
        
#     y_test_majority = []
#     for i in range(0, y_data.shape[0], 5):
#         segment_predictions = y_test[i:i+5]
#         majority_vote = np.bincount(segment_predictions).argmax()
#         y_test_majority.append(majority_vote)    
    
#     sum=sum+accuracy_score(y_test_majority,y_pred_majority)
#     print(confusion_matrix(y_test_majority,y_pred_majority))
#     print(accuracy_score(y_test_majority,y_pred_majority))
# print(sum/5)

In [ ]:
# sum=0
# for tr_idx, test_idx in kf.split(xt):
#     X_train, X_test=xt.iloc[tr_idx,:],xt.iloc[test_idx,:]
#     y_train, y_test=y_data[tr_idx],y_data[test_idx]

#     classifier=SGDClassifier()
#     classifier.fit(X_train,y_train)

#     ypred=classifier.predict(X_test)
#     sum=sum+accuracy_score(y_test,ypred)
#     print(confusion_matrix(y_test,ypred))
#     print(accuracy_score(y_test,ypred))
# print(sum/5)

In [ ]:
# y_pred=classifier.predict(X_test)

In [ ]:
# y_pred_majority = []
# for i in range(0, y_data.shape[0], 5):
#     segment_predictions = y_pred[i:i+5]
#     majority_vote = np.bincount(segment_predictions).argmax()
#     y_pred_majority.append(majority_vote)

In [ ]:
# y_test_majority = []
# for i in range(0, y_data.shape[0], 5):
#     segment_predictions = y_test[i:i+5]
#     majority_vote = np.bincount(segment_predictions).argmax()
#     y_test_majority.append(majority_vote)

In [ ]:
# from sklearn.metrics import confusion_matrix

In [ ]:
# accuracy = accuracy_score(y_test_majority, y_pred_majority)
# print(accuracy)

In [ ]:
# print(confusion_matrix(y_test,ypred))
# print(accuracy_score(y_test,ypred))

In [ ]:
xt.shape